<a href="https://colab.research.google.com/github/MarcusPauloFaustino1/Exercicios-Growdev/blob/main/Trabalho_Final_Prog_DS_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importações

import pandas as pd
import numpy as np
pd.options.plotting.backend = 'plotly'
import csv

# leitura de arquivos

pokemon = pd.read_csv('pokemon.csv')

pokemon_tipos_1 = pd.read_csv('pokemon_tipos_1.csv')

pokemon_tipos_2 = pd.read_csv('pokemon_tipos_2.csv')

# informações sobre arquivos

print(pokemon.head(10)) # A coluna "#" domonstrou-se desnecessária 

print(pokemon.info()) # 414 pokemons possuem apenas o tipo primário
                      # Não encontrou-se inconsistências

print("\n\n")

print(pokemon_tipos_1.info())

print("\n\n")

print(pokemon_tipos_2.info()) 

print(pokemon.describe()) # Obsevando o describe, não há outliers.

duplicados = pokemon[pokemon.duplicated(keep='first')] # Não há dados duplicados

duplicados.info()




## **1) Verifique se há a necessidade de realizar algum tratamento na base de dados, caso julgue necessário realize o tratamento e comente no código o porquê de estar realizando determinada técnica de limpeza.**

In [ ]:
# Tratamento de dados

pokemon.drop(['#'],axis = 1, inplace = True) # excluindo a coluna "#", por julgá-la desnecessária.


pokemon['pokemon_tipo_2_id'] = pokemon['pokemon_tipo_2_id'].fillna(0) # preenchendo NaN para 0, pois quase metade dos pokemons não possuem tipo_2, 
                                                                      # logo não vale a pena excluí-la, outro motivo é para poder convertê-la em tipo int.


pokemon['pokemon_tipo_2_id'] = pokemon['pokemon_tipo_2_id'].astype('int16') # alterando tipo de dado, de float64 para int16, com objetivo de otimizar o gasto de memória.

pokemon_tipos_2.loc[18] = [0,'-'] # adicionando novo nome para pokemons que não possuem tipo secundário

pokemon = pokemon.astype({col: 'int16' for col in pokemon.select_dtypes('int64').columns}) # convertendo int64 para int16

print(pokemon.info()) # conferindo informações atualizadas

,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario
0,Bulbasaur,10,14,318,45,49,49,65,65,45,1,False
1,Ivysaur,10,14,405,60,62,63,80,80,60,1,False
2,Venusaur,10,14,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,10,14,625,80,100,123,122,120,80,1,False
4,Charmander,7,0,309,39,52,43,60,50,65,1,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   pokemon_nome             800 non-null    object
 1   pokemon_tipo_1_id        800 non-null    int16 
 2   pokemon_tipo_2_id        800 non-null    int16 
 3   pokemon_total            800 non-null    int16 
 4   pokemon_hp               800 non-null    int16 
 5   pokemon_attack           800 non-null    int16 
 6   pokemon_defense          800 non-null    int16 
 7   pokemon_ataque_especial  800 non-null    int16 
 8   pokemon_defesa_especial  800 non-null    int16 
 9   pokemon_velocidade       800 non-null    int16 
 10  pokemon_geracao          800 non-null    int16 
 11  pokemon_lendario         800 non-null    bool  
dtypes: bool(1), int16(10), object(1)
memory usage: 22.8+ KB
None


## **2) Procure os pokémons do tipo Grass que possuem mais de 100 de ataque e são da geração 3.**

In [ ]:
pkm_todos_tipos = pokemon.merge(pokemon_tipos_1, how = 'outer').merge(pokemon_tipos_2, how = 'outer') # unindo as tabelas do tipo_1 e do tipo_2


pkm_todos_tipos.query('pokemon_attack > 100 & pokemon_geracao == 3 '
                      '& (pokemon_tipo_1_nome == "Grass"'
                      '| pokemon_tipo_2_nome == "Grass")') # Considerou-se para o tipo Grass,tanto o tipo_1 quanto o tipo_2




,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario,pokemon_tipo_1_nome,pokemon_tipo_2_nome
550,SceptileMega Sceptile,10,3,630,70,110,75,145,85,145,3,False,Grass,Dragon
570,Cacturne,10,2,475,70,115,60,115,60,55,3,False,Grass,Dark
588,Breloom,10,6,460,60,130,80,60,60,70,3,False,Grass,Fighting


## **3) Faça um comparativo entre os tipos de pokémon e as gerações, evidencie em cada tipo quais gerações possuem mais de 50 pokémons. Dica: use pivoteamento.**

In [207]:

def soma(values):         
    soma_valores = values.sum()   # definindo função de soma
    return soma_valores

pokemons_tipo_geracao_1 = pd.pivot_table(pkm_todos_tipos,
    index=['pokemon_tipo_1_nome'],  
    columns=['pokemon_geracao'],                              # pivoteamento voltado para pokemons do tipo_1, de forma que construa uma tabela contendo como índice
    aggfunc={'pokemon_geracao': soma},                        # o tipo_1 e como colunas as somas dos indivíduos por geração.
    fill_value =0
)

pokemons_tipo_geracao_2 = pd.pivot_table(pkm_todos_tipos,
    index=['pokemon_tipo_2_nome'],                            # pivoteamento voltado para pokemons do tipo_2, de forma que construa uma tabela contendo como índice
    columns=['pokemon_geracao'],                              # o tipo_2 e como colunas as somas dos indivíduos por geração.
    aggfunc={'pokemon_geracao': soma},
    fill_value =0
)

df_total = pokemons_tipo_geracao_1['pokemon_geracao'] + pokemons_tipo_geracao_2['pokemon_geracao'].drop(["-"])  # Considerou-se o tipo_1 e o tipo_2 dos pokemons, de forma
                                                                                                                # que cada um deles pode contar como um ou dois tipos 
                                                                                                                # ao mesmo tempo em cada geração.
                                                                                                                # E também excluindo os tipos de número "0".


df_total.where(df_total > 50, 0)   # Selecionando os valores maiores que 50, e denominando como "0" o que fosse diferente disso.


pokemon_geracao,1,2,3,4,5,6
pokemon_tipo_1_nome,,,,,,
Bug,0,0,0,0,90,0
Dark,0,0,0,0,80,0
Dragon,0,0,0,0,60,54
Electric,0,0,0,0,60,0
Fairy,0,0,0,0,0,84
Fighting,0,0,0,0,85,0
Fire,0,0,0,0,80,0
Flying,0,0,0,64,105,0
Ghost,0,0,0,0,0,90


## **4) Faça um comparativo entre as gerações e a coluna de pokémons lendários para verificar quantos pokémons existem com velocidade acima de 85. Dica: use pivoteamento.**

In [98]:
pokemon_velocidade_maior_85 = pkm_todos_tipos.query('pokemon_velocidade > 85')   # Selecionando pokemon com velocidade superior a 85

pokemons_lendarios_velocidade = pd.pivot_table(pokemon_velocidade_maior_85,
              index = ['pokemon_geracao'],                                       # pivoteamento para construir uma tabela contendo como índice 
              columns=['pokemon_lendario'],                                      # as gerações e como colunas a soma do número de pokemons lendários
              aggfunc={'pokemon_geracao': np.size})                              # presentes em cada geração.

display(pokemons_lendarios_velocidade)

pokemon_geracao     
pokemon_lendario           False True
pokemon_geracao                      
1                             49    5
2                             14    4
3                             25   15
4                             32   11
5                             37   15
6                             14    4

## **5) Plote um gráfico mostrando o valor somado da coluna pokemon_total por tipo de pokemon, contando apenas os pokemons que possuem defesa menor que 50.**

In [160]:
# Criando uma nova coluna chamada "tipo_definitivo", para tornar possível, de uma única vez, a classificação dos pokemons por tipo e subtipo.

pkm_todos_tipos['tipo_definitivo'] = pkm_todos_tipos['pokemon_tipo_1_nome'].astype('str') + '/' + pkm_todos_tipos['pokemon_tipo_2_nome'].astype('str')


def soma(values):
    soma_valores = values.sum()   # Criando função para soma de valores.
    return soma_valores

pokemons_defesa_menor_50_tipo = pkm_todos_tipos.query('pokemon_defense < 50')    # Selecionando pokemons com defesa menor que 50.

pokemons_total_tipo_defesa = pd.pivot_table(pokemons_defesa_menor_50_tipo,
                        index = ['tipo_definitivo'],                        #  Pivoteamento para criar tabela tendo com index os tipos definitivos
                        aggfunc={'pokemon_total': soma}                     # e como colunas a soma dos valores de pokemon_total de todos tipos definitivos de pokemons.
                        )

pokemons_total_tipo_defesa.plot(kind='bar')  # plotando o gráfico



In [170]:
'''6) Faça uma comparação entre pokemons do tipo fire e do tipo grass
evidenciando qual tem mais defesa no total e qual tem mais ataque
no total.'''

pkm_fire_grass_1 = pkm_todos_tipos.query('pokemon_tipo_1_nome == "Grass" | pokemon_tipo_1_nome == "Fire"')

pkm_fire_grass_2 = pkm_todos_tipos.query('pokemon_tipo_2_nome == "Grass" | pokemon_tipo_2_nome == "Fire"')

pkm_fire_grass_atk_def_1_sum = pd.pivot_table(pkm_fire_grass_1, index = ['pokemon_tipo_1_nome'],aggfunc= {'pokemon_attack': np.sum,'pokemon_defense': np.sum})

pkm_fire_grass_atk_def_2_sum = pd.pivot_table(pkm_fire_grass_2, index = ['pokemon_tipo_2_nome'],aggfunc= {'pokemon_attack': np.sum,'pokemon_defense': np.sum})

pkm_fire_grass_atk_def_sum_final = pkm_fire_grass_atk_def_1_sum + pkm_fire_grass_atk_def_2_sum

pkm_fire_grass_atk_def_1_mean = pd.pivot_table(pkm_fire_grass_1, index = ['pokemon_tipo_1_nome'],aggfunc= {'pokemon_attack': np.mean,'pokemon_defense': np.mean})

pkm_fire_grass_atk_def_2_mean = pd.pivot_table(pkm_fire_grass_2, index = ['pokemon_tipo_2_nome'],aggfunc= {'pokemon_attack': np.mean,'pokemon_defense': np.mean})

print(f'\n\n\nSomatório de ataque e defesa dos pokemons tipo Grass e tipo Fire: ')

display(pkm_fire_grass_atk_def_sum_final)

pkm_fire_grass_atk_def_mean_final = (pkm_fire_grass_atk_def_1_mean +pkm_fire_grass_atk_def_2_mean)/2

print(f'\n\n\nMédia de ataque e defesa dos pokemons tipo Grass e tipo Fire: ')

display(pkm_fire_grass_atk_def_mean_final)




Somatório de ataque e defesa dos pokemons tipo Grass e tipo Fire: 


,pokemon_attack,pokemon_defense
pokemon_tipo_1_nome,,
Fire,5383,4486
Grass,6979,6959





Média de ataque e defesa dos pokemons tipo Grass e tipo Fire: 


,pokemon_attack,pokemon_defense
pokemon_tipo_1_nome,,
Fire,83.009615,73.967949
Grass,73.687143,75.460000


In [206]:
'''7) Um treinador pokémon quer escolher um pokémon para a batalha.
Faça uma análise para indicar o melhor pokemon possível para o
embate.
a) O adversário possui um pokémon do tipo Grass, e que esse
tipo possui desvantagem sobre pokémons do tipo Bug, Fire,
Flying, Ice e Poison.
b) O pokémon deve ser no máximo da geração 3
c) O ataque do pokémon deve ser menor ou igual a 100.

d) A defesa deve ser maior que 30.
e) O pokémon pode ter um tipo secundário que não influencia na
regra a.
f) O pokémon não pode ser lendário.'''

pkm_tipos = pkm_todos_tipos.query(
    '(pokemon_tipo_1_nome == "Bug"'
    ' | pokemon_tipo_1_nome == "Fire"'
    ' | pokemon_tipo_1_nome == "Flying"'
    ' | pokemon_tipo_1_nome == "Ice"'
    ' | pokemon_tipo_1_nome == "Poison")'
    ' & pokemon_lendario == False'
    ' & pokemon_geracao <= 3'
    ' &pokemon_defense > 30 '
    ' & pokemon_attack <= 100')

pkm_tipos['pokemon_total'].max()

pkm_total = pkm_tipos.query('pokemon_total == 535')

pkm_total

,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario,pokemon_tipo_1_nome,pokemon_tipo_2_nome,tipo_definitivo
511,Crobat,14,8,535,85,90,80,70,80,130,2,False,Poison,Flying,Poison/Flying
